In [2]:
import pandas as pd
from curl_cffi import requests as cureq
from bs4 import BeautifulSoup
import re
import os

In [3]:
from pydantic import BaseModel
from typing import List
import time

In [7]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
}

class JobListing(BaseModel):
    jobLink: List[str]
    jobTitle: List[str]
    jobCompany: List[str]
    minSalary: List[str]
    maxSalary: List[str]
    jobDetails: List[str]
    jobLocation: List[str]

def extract_text(element, default='None'):
    """Helper function to extract text or return a default value."""
    return element.get_text().strip() if element else default

def extract_salary(job_salary):
    """Helper function to split and clean salary into min and max."""
    if job_salary != 'Not Specified' and len(job_salary.split(' ')) > 2:
        min_salary = job_salary.split('-')[0].replace('$', '').strip()
        max_salary = job_salary.split('-')[1].split(' ')[1].replace('$', '').strip()
        return min_salary, max_salary
    return 'None', 'None'

def pull_job_details(resp):
    job_list = {'jobLink':[],'jobTitle':[],'jobCompany':[],'minSalary':[],'maxSalary':[],'jobDetails':[],'jobLocation':[]}

    if 'text/html' in resp.headers['Content-Type'] and resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')

        outer_most_point=soup.find('div',attrs={'id': 'mosaic-provider-jobcards'})

        for job in outer_most_point.find('ul'):
            a = job.find('a')
            if not a:
                continue

            href_link = a.get('href')
            job_link = 'https://www.indeed.com' + href_link
            job_list['jobLink'].append(job_link)

            job_salary,job_description = pull_job_desc(job_link)
 
            min_salary, max_salary = extract_salary(job_salary)

            job_list['minSalary'].append(min_salary)
            job_list['maxSalary'].append(max_salary)
            job_list['jobDetails'].append(job_description)

            job_list['jobTitle'].append(
                extract_text(job.find('span', id=lambda x: x and x.startswith('jobTitle-')))
            )
            job_list['jobCompany'].append(
                extract_text(job.find('span', {'data-testid': 'company-name'}))
            )
            job_list['jobLocation'].append(
                extract_text(job.find('div', {'data-testid': 'text-location'}))
            )

    return job_list

def pull_job_desc(job_link):
    resp = cureq.get(job_link,impersonate='chrome')

    if 'text/html' in resp.headers['Content-Type'] and resp.status_code == 200:

        soup = BeautifulSoup(resp.text,'html.parser')
        outer_most_points = soup.find('div',class_=re.compile(r'^fastviewjob'))

        raw_salary = outer_most_points.find('div',attrs={'id':'salaryInfoAndJobType'})
        salary = 'Not Specified'
        
        if raw_salary:
            salary = raw_salary.get_text()
        
        raw_description = outer_most_points.find('div',attrs={'id':'jobDescriptionText'})
        description = 'None'
        
        if raw_description:
            description = raw_description.get_text().replace('\n','')
        
    return salary,description

def format_search(search):
    return search.replace(' ','+')

def new_session():
    session = cureq.Session(impersonate="chrome",proxy=os.getenv("stickyproxy"))
    return session

def search_api(session: cureq.Session, job_title: str, location:str, start_num: int):
    url = f"https://www.indeed.com/jobs?q={format_search(job_title)}&l={format_search(location)}%2C++CA&start={str(start_num)}"
    resp = session.get(url,headers=headers)
    resp.raise_for_status()
    return JobListing(**pull_job_details(resp))

### Use loop to pull all current job listings with a search query

In [8]:
df = pd.DataFrame()
for i in range(0,1):
    listing = search_api(new_session(),'data+engineer','mountain+view',i*10)
    df = pd.concat([df,pd.DataFrame(listing.dict())],ignore_index=True)
    time.sleep(2)

In [9]:
df

,jobLink,jobTitle,jobCompany,minSalary,maxSalary,jobDetails,jobLocation
0,/rc/clk?jk=5730c9f7bbab0b75&bb=D8e4_gNEhqXZdqP...,Associate Data Engineer,Analog Devices,"78,200","107,525",Come join Analog Devices (ADI) – a place whe...,"San Jose, CA 95134"
1,/rc/clk?jk=91f171f8260d5452&bb=D8e4_gNEhqXZdqP...,Data Engineer,Analog Devices,"97,060","133,458",Come join Analog Devices (ADI) – a place whe...,"San Jose, CA 95134"
2,/rc/clk?jk=afe0d997c69eac80&bb=D8e4_gNEhqXZdqP...,Data Engineer II,Databricks,"92,800","164,200","GAQ425R73 Location: San Francisco, CA or Mount...","Mountain View, CA"
3,/rc/clk?jk=6983e393d1853893&bb=D8e4_gNEhqXZdqP...,"New College Graduate, Data Engineer (May 2024 ...",Western Digital,None,None,"Company Description At Western Digital, our vi...","San Jose, CA 95119"
4,/rc/clk?jk=853c03f5baacd62c&bb=D8e4_gNEhqXZdqP...,Analytics Engineer (L5) - Member Data Products,Netflix,"170,000","720,000",Job Requisition ID JR30168 Job Posting...,"Los Gatos, CA"
5,/rc/clk?jk=df819344e3d7063b&bb=D8e4_gNEhqXZdqP...,"Data Engineer, UDW",Samsung Electronics,None,None,Position Summary Technical professionals ar...,"Mountain View, CA 94043"
6,/rc/clk?jk=95f7bd3fb1e18475&bb=D8e4_gNEhqXZdqP...,Data Engineer,Stanford University,"96,000","140,000","(Full-time, 2 year contract, renewable)The Sta...","Stanford, CA"
7,/rc/clk?jk=4f52cc770d2bb30d&bb=D8e4_gNEhqXZdqP...,Data Engineer,UST Global,None,None,1 Opening San Jose Role de...,"San Jose, CA"
8,/rc/clk?jk=a5c104503914b1c6&bb=D8e4_gNEhqXZdqP...,Big Data Infrastructure Engineer,Zoom,"118,600","259,500",Immigration sponsorship is not available for ...,"Hybrid work in San Jose, CA 95113"
9,/rc/clk?jk=14ff6f2b78889877&bb=D8e4_gNEhqXZdqP...,SSP Data User Studies Research Engineer,Apple,"136,300","248,700","Summary Posted: Sep 20, 2024 Role Number:200...","Cupertino, CA"


In [155]:
def clean_output(text:str):
    text = text.lower()
    text = text.replace('\n','    ')
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    return text

def pull_skills(resp):
    if 'text/html' in resp.headers['Content-Type'] and resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        print(soup)

        # Find all button elements with 'data-testid' that ends with '-tile'
        buttons = soup.find_all('button', {'data-testid': lambda x: x and x.endswith('-tile')})
        # Extract the tile name from the 'data-testid' attribute
        tile_names = [button['data-testid'].replace('-tile', '') for button in buttons]

        for name in tile_names:
            print(name)

def pull_job_data(resp):
    if 'text/html' in resp.headers['Content-Type'] and resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')

        desc = soup.find('div',{'id':'jobDescriptionTitle'})

        if not desc:
            print('No job description found')
            return None
        
        return desc.next_sibling.get_text()
        
def get_raw_text(resp):
    soup = BeautifulSoup(resp.text,'html.parser')
    return clean_output(soup.get_text())

def scrape_webpage_text(session: cureq.Session, job_link: str):
    url = f'https://www.indeed.com{job_link}'
    resp = session.get(url,headers=headers)
    
    return resp

### Update to pull job skills if necessary

In [156]:
for index,row in df.iterrows():
    resp = scrape_webpage_text(new_session(),row['jobLink'])
    desc = clean_output(pull_job_data(resp))
    df.loc[index, 'description'] = desc
    time.sleep(2)

In [157]:
df.head(5)

,jobLink,jobTitle,jobCompany,minSalary,maxSalary,jobDetails,description
0,/rc/clk?jk=91f171f8260d5452&bb=Eb6IIhKxqsmRoSA...,Data Engineer,Analog Devices,"97,060","133,458",Come join Analog Devices (ADI) – a place whe...,come join analog devices adi a plac...
1,/rc/clk?jk=853c03f5baacd62c&bb=Eb6IIhKxqsmRoSA...,Analytics Engineer (L5) - Member Data Products,Netflix,"170,000","720,000",Job Requisition ID JR30168 Job Posting...,job requisition id ...
2,/rc/clk?jk=5730c9f7bbab0b75&bb=Eb6IIhKxqsmRoSA...,Associate Data Engineer,Analog Devices,"78,200","107,525",Come join Analog Devices (ADI) – a place whe...,come join analog devices adi a plac...
3,/rc/clk?jk=4f52cc770d2bb30d&bb=Eb6IIhKxqsmRoSA...,Data Engineer,UST Global,None,None,1 Opening San Jose Role de...,1 opening ...
4,/rc/clk?jk=95f7bd3fb1e18475&bb=Eb6IIhKxqsmRoSA...,Data Engineer,Stanford University,"96,000","140,000","(Full-time, 2 year contract, renewable)The Sta...",fulltime 2 year contract renewable the ...
